# KDDxy

In [4]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [5]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')   
try:
    poimanagerlogic
except NameError:
    manager.startModule('logic', 'poimaganerlogic')    

In [6]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [22]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV2'
setup['poi'] = poimanagerlogic.active_poi.get_key()
setup['laser_length'] = 3.0e-6
setup["min_counts"] = 10000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

In [39]:
# for single frequency XY8-N:
kdd = OrderedDict()
kdd['mw_amp'] = 0.25
kdd['mw_freq'] = 100e6
kdd['order'] = 28
kdd['tau_start'] = 206e-9
kdd['tau_incr'] = 1e-9
kdd['points'] = 21
kdd['measurement_time'] = 6*60
kdd['refocus_interval'] = 2*60
kdd['alternating']=True
kdd['rabi_period']=143.0*1e-9

In [40]:
# generate new KDD sequence
sequencegeneratorlogic.delete_sequence('KDDxy_sequence')    
sequencegeneratorlogic.generate_KDDxy_sequence('KDDxy_sequence', kdd['rabi_period'], 100e+6, 0.25,
                          kdd['tau_start'], kdd['tau_incr'], kdd['points'], kdd['order'],
                          'a_ch1', 3.0e-6, 2.0, 0.7e-6, 1.0e-6, '', 'd_ch2', True)
# sample, upload and load waveform
if 'KDDxy_sequence' not in sequencegeneratorlogic.saved_pulse_sequences:
    sequencegeneratorlogic.log.error('Sequence generation of KDD in notebook timed out.')
pulsedmasterlogic.sample_sequence('KDDxy_sequence', True)

while pulsedmasterlogic.status_dict['sauplo_sequence_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(5)
        
import winsound
frequency = 440  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [41]:
setup['poi'] = poimanagerlogic.active_poi.get_key()
refocus_poi(setup['poi'])

179971.333333 1.1764432892234203e-08


True

In [26]:
do_kdd_refocus(setup['poi'])

179971.333333 4.077654191944751e-08
179971.333333 9.516864374816157e-09
179971.333333 1.8544792477432422e-10
179971.333333 8.17880817020574e-10
179971.333333 8.237709788072334e-09
179971.333333 1.0092203692517942e-09
179971.333333 3.2600926226608086e-09
179971.333333 1.2149782831732796e-09
179971.333333 1.4050755307580304e-09
179971.333333 9.32530145551086e-10
179971.333333 3.3201506001762798e-09
179971.333333 1.41869854409623e-09
179971.333333 4.6158653859634596e-09
179971.333333 6.341715439405187e-09


False

In [27]:
# save measurement
pulsedmasterlogic.save_measurement_data('s','kdd-'+str(kdd['order'])+'_'+setup['NV_name'],True)

In [42]:
do_kdd('kdd_refocus','')

True

In [ ]:
refocus_poi(setup['poi'])
do_kdd('kdd_refocus', 'kdd_refocus')

In [8]:
def refocus_poi(poi=None, max_drift=0.3,min_counts=setup["min_counts"]):
    if poi==None:
        return False
#     pulsedmasterlogic.load_asset_into_channels('Laser_On')
#     while pulsedmasterlogic.status_dict['loading_busy']:
#         time.sleep(0.5)
#     pulsedmasterlogic.toggle_pulse_generator(False)
#     time.sleep(0.5)
    # perform refocus
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.getState() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.getState() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

def do_kdd(save_tag='',load_tag=''):
    
    pulsedmasterlogic.do_fit('No Fit')
    tau_arr = kdd['tau_start'] + np.arange(kdd['points']) * kdd['tau_incr']
    # set parameters in analysis tab
    if kdd['alternating']:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, 2*kdd['points'], 100e-6, [],kdd['alternating'])
    else:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, kdd['points'], 100e-6, [], kdd['alternating'])
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+0.2e-6)
    pulsedmasterlogic.analysis_interval_changed(1)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < kdd['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data('s','KDD-'+str(kdd['order'])+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

def do_kdd_refocus(poi):
    end_measure=False
    #refocus_poi(poi)
    kdd_total_time = kdd['measurement_time']
    kdd_runtime = 0.0
    kdd['measurement_time'] = kdd['refocus_interval']
    end_measure = do_kdd('kdd_refocus','')
    kdd_runtime += kdd['refocus_interval']
    while kdd_total_time > kdd_runtime:
        for i in range(1):
            end_measure= not refocus_poi(poi)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_kdd('kdd_refocus', 'kdd_refocus')
        kdd_runtime += kdd['refocus_interval']
        if end_measure:
            break
    kdd['measurement_time'] = kdd_total_time
#     save measurement
#     pulsedmasterlogic.save_measurement_data('s','kdd-'+str(kdd['order'])+'_'+setup['NV_name'],True)
    write_to_logfile('kdd_log', str(datetime.datetime.now()),name= setup['NV_name'],runtime=kdd_runtime)
    time.sleep(2)
    return end_measure